In [1]:
import numpy as np
import matplotlib.pyplot as plt
import energyflow as ef
import energyflow.archs
from energyflow.archs import PFN
from matplotlib import gridspec
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Layer, concatenate, ReLU
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.layers import BatchNormalization
from scipy.stats import norm
from scipy.stats import zscore
from keras import backend

plt.rc('font', size=20)
plt.rcParams["font.family"] = "serif"

In [2]:
normalize = True
data_size = 10**6
mylambda = []

In [3]:
datasets = {'Pythia26': ef.zjets_delphes.load('Pythia26', num_data=data_size),
            'Herwig': ef.zjets_delphes.load('Herwig', num_data=data_size)}

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [4]:
w_gen = datasets['Pythia26']['gen_widths'] #gen = particle level
w_sim = datasets['Pythia26']['sim_widths'] #sim = detector level
w_truth = datasets['Herwig']['gen_widths']
w_data = datasets['Herwig']['sim_widths']

p_gen = datasets['Pythia26']['gen_jets'][:,0] #gen = particle level
p_sim = datasets['Pythia26']['sim_jets'][:,0] #sim = detector level
p_truth = datasets['Herwig']['gen_jets'][:,0]
p_data = datasets['Herwig']['sim_jets'][:,0]


if normalize:   
    wm = np.mean(w_truth)
    ws = np.std(w_truth)
    pm = np.mean(p_truth)
    ps = np.mean(p_truth)
    
    w_gen = (w_gen - wm)/ws
    w_sim = (w_sim - wm)/ws
    w_truth = (w_truth - wm)/ws
    w_data = (w_data - wm)/ws
    
    p_gen = (p_gen - pm)/ps
    p_sim = (p_sim - pm)/ps
    p_truth = (p_truth - pm)/ps
    p_data = (p_data - pm)/ps

In [5]:
def weighted_binary_crossentropy(y_true, y_pred):
    weights = tf.gather(y_true, [1], axis=1) # event weights
    y_true = tf.gather(y_true, [0], axis=1) # actual y_true for loss

    weights_1 = K.sum(y_true*weights)
    weights_0 = K.sum((1-y_true)*weights)

    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = -weights * ((y_true) * K.log(y_pred)/weights_1 +
                         (1 - y_true) * K.log(1 - y_pred)/weights_0)
    return K.mean(t_loss)

def weighted_binary_crossentropy_GAN(y_true, y_pred):
    weights = tf.gather(y_pred, [1], axis=1) # event weights
    y_pred = tf.gather(y_pred, [0], axis=1) # actual y_pred for loss

    weights_1 = K.sum(y_true*weights)
    weights_0 = K.sum((1-y_true)*weights)

    #tf.print("weights",weights_0,weights_1)

    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
    t_loss = weights * ((1 - y_true) * K.log(1 - y_pred)/weights_0)
    return K.mean(t_loss)

In [6]:
@tf.keras.utils.register_keras_serializable(package='Custom', name='entropy')
class EntropyRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, beta=0.):
        self.beta = beta

    def __call__(self, x):
        return -1*self.beta * tf.math.reduce_sum(-1*x*tf.math.log(x))

    def get_config(self):
        return {'beta': float(self.beta)}

In [19]:
mymodel_inputtest = Input(shape=(1,))
hidden_layer_1 = Dense(50, activation='ReLU',use_bias=False)(mymodel_inputtest)
batch_norm_1 = BatchNormalization()(hidden_layer_1)
hidden_layer_2 = Dense(50, activation='ReLU',use_bias=False)(batch_norm_1)
batch_norm_2 = BatchNormalization()(hidden_layer_2)
hidden_layer_3 = Dense(50, activation='ReLU',use_bias=False)(batch_norm_2)
outputs = tf.exp(Dense(1, activation='sigmoid',use_bias=False,
                      kernel_regularizer=EntropyRegularizer(beta = 0.5))(hidden_layer_3))
model_generator = Model(mymodel_inputtest, outputs)

inputs_disc = Input((1, ))
hidden_layer_1_disc = Dense(50, activation='relu')(inputs_disc)
hidden_layer_2_disc = Dense(50, activation='relu')(hidden_layer_1_disc)
hidden_layer_3_disc = Dense(50, activation='relu')(hidden_layer_2_disc)
outputs_disc = Dense(1, activation='sigmoid')(hidden_layer_3_disc)
model_discrimintor = Model(inputs=inputs_disc, outputs=outputs_disc)

model_discrimintor.compile(loss=weighted_binary_crossentropy, optimizer='adam')

model_discrimintor.trainable = False
mymodel_gan = Input(shape=(1,))
gan_model = Model(inputs=mymodel_gan,outputs=concatenate([model_discrimintor(mymodel_gan),model_generator(mymodel_gan)]))


gan_model.compile(loss=weighted_binary_crossentropy_GAN, optimizer='adam')

In [20]:
xvals_particle = np.concatenate([w_gen,w_truth])
xvals_detector = np.concatenate([w_sim,w_data])     
yvals = np.transpose(np.concatenate([np.ones(len(w_gen)),np.zeros(len(w_truth))]))

X_train_particle, X_test_particle, X_train_detector, X_test_detector, Y_train, Y_test = train_test_split(xvals_particle, 
                                                                                                        xvals_detector,
                                                                                                        yvals)

In [ ]:
n_epochs = 20
n_batch = 128*10
n_batches = len(X_train_particle) // n_batch

for i in range(n_epochs):
    mypreds = model_generator.predict(X_test_particle,batch_size=1000)
    print("on epoch=",i,np.mean(mypreds),np.min(mypreds),np.max(mypreds))
    for j in range(n_batches):
        X_batch_particle = X_train_particle[j*n_batch:(j+1)*n_batch]
        X_batch_detector = X_train_detector[j*n_batch:(j+1)*n_batch]
        Y_batch = Y_train[j*n_batch:(j+1)*n_batch]
        W_batch = model_generator(X_batch_particle)
        W_batch = np.array(W_batch).flatten()

        W_batch[Y_batch==1] = 1        
        Y_batch_2 = np.stack((Y_batch, W_batch), axis=1)

        model_discrimintor.train_on_batch(X_batch_detector, Y_batch_2)        
        gan_model.train_on_batch(X_batch_particle[Y_batch==0],Y_batch[Y_batch==0])

on epoch= 0 1.6476638 1.6285193 1.6495969
on epoch= 1 nan nan nan
on epoch= 2 nan nan nan
on epoch= 3 nan nan nan
on epoch= 4 nan nan nan
on epoch= 5 nan nan nan
on epoch= 6 nan nan nan
on epoch= 7 nan nan nan
on epoch= 8 nan nan nan


In [ ]:
weights = model_generator.predict(X_test_particle)

In [ ]:
#bins = np.linspace(0,1,30)
fig, ax = plt.subplots(figsize=(8, 6))

nbins = 30
bins = np.linspace(X_test_particle.min(),X_test_particle.max(),nbins)

_,_,_=plt.hist(X_test_particle[Y_test==0],bins=bins,alpha=0.5,label="truth",density=True, zorder=-1)
_,_,_=plt.hist(X_test_particle[Y_test==1],bins=bins,alpha=0.5,label="gen",density=True, zorder=0)
_,_,_=plt.hist(X_test_particle[Y_test==0],bins=bins,weights=weights[Y_test==0],histtype="step",color="red",ls=":", lw=2,label="weighted gen",density=True, zorder=1)
plt.legend(fontsize=15)
plt.ylabel("Trials")
plt.xlabel("jet width")
#plt.savefig("Unifold.pdf", bbox_inches='tight', transparent=True)
plt.show()